In [ ]:
# Generate random SSYTs for a given value of (n, k, rank)
import random, multiprocessing

n = 12
k = 3
rank = 5
samples = 45000

def sample_ssyt(samples, piece):
    with open(f"Data/{k}_{rank}_{n}_random.{piece}.txt", "w") as f:
        set_random_seed(piece)
        S = SemistandardTableaux([rank]*k, max_entry=n)
        while samples > 0:
            f.write(str(S.random_element())+"\n")
            samples -= 1

pieces = multiprocessing.cpu_count()
procs = []
for i in range(pieces):
    proc = multiprocessing.Process(target=sample_ssyt, args=(samples,i))
    procs.append(proc)
    proc.start()

for p in procs:
    p.join()

with open(f"Data/{k}_{rank}_{n}_random.txt", "w") as f:
    for i in range(pieces):
        with open(f"Data/{k}_{rank}_{n}_random.{i}.txt", "r") as rf:
            f.write(rf.read())

In [1]:
import itertools

n = 3
m = 8
max_rank = 3

A = CartanMatrix(['A', n-1]).inverse()

M = list(range(1, m+1))

# The possible rank 1 SSYTs are the possible partitions of [m] into n parts.
columns = []
column_idx = {}
# The weight of an SSYT is stored as a tuple representing a non-negative
# linear combination of simple roots.
#
# Keys are column indexes.
column_weights = {}

for column in itertools.combinations(range(m), n):
    column_idx[column] = len(columns)
    columns.append(column)

    # Weight computation using the inverse Cartan matrix.
    wt = [0 for _ in range(n-1)]
    for i in range(n-1):
        d = column[i+1] - column[i] - 1
        col = d*A.column(n-1-(i+1)).row()
        for j in range(n-1):
            wt[j] += col[0][j]
    column_weights[column_idx[column]] = tuple(wt)

# An SSYT is a tuple of columns. Instead of storing the columns, we store the
# # column indexes.
ssyts = []
ssyt_idx = {}
ssyt_weights = {}

# Add an ssyt. Returns the index of the ssyt.
def add_ssyt(ssyt):
    if ssyt in ssyt_idx:
        return ssyt_idx[ssyt]
    ssyt_idx[ssyt] = len(ssyts)
    ssyts.append(ssyt)
    wt = [0 for _ in range(n-1)]
    for i in ssyt:
        for j in range(n-1):
            wt[j] += column_weights[i][j]
    ssyt_weights[ssyt_idx[ssyt]] = tuple(wt)
    return ssyt_idx[ssyt]

# Returns ssyt1 + ssyt2, where ssyt1 and ssyt2 are tuples of column indexes.
def union_ssyt(*ssyts):
    ssyts = [[columns[i] for i in ssyt] for ssyt in ssyts]
    rows = [sorted(sum([[ssyt[c][r] for c in range(len(ssyt))] for ssyt in ssyts], [])) for r in range(n)]
    union_ssyt = [[rows[r][c] for r in range(n)] for c in range(len(rows[0]))]
    return tuple(column_idx[tuple(col)] for col in union_ssyt)

def repeat_ssyt(ssyt, k):
    new_ssyt = []
    for c in ssyt:
        for i in range(k):
            new_ssyt.append(c)
    return new_ssyt


    # union_ssyt = [sorted([ssyt[i][0 for _ in range(n)] for i in range(sum([len(ssyt) for ssyt in ssyts]))]
    # for i in range(n):
    #     # Linear time union of two sorted lists by mainting pointers.
    #     idx1 = 0
    #     idx2 = 0
    #     while idx1 < len(ssyt1) or idx2 < len(ssyt2):
    #         if idx1 == len(ssyt1):
    #             union_ssyt[idx1+idx2][i] = ssyt2[idx2][i]
    #             idx2 += 1
    #         elif idx2 == len(ssyt2):
    #             union_ssyt[idx1+idx2][i] = ssyt1[idx1][i]
    #             idx1 += 1
    #         elif ssyt1[idx1][i] < ssyt2[idx2][i]:
    #             union_ssyt[idx1+idx2][i] = ssyt1[idx1][i]
    #             idx1 += 1
    #         else:
    #             union_ssyt[idx1+idx2][i] = ssyt2[idx2][i]
    #             idx2 += 1
    # return tuple(column_idx[tuple(col)] for col in union_ssyt)

# Returns ssyt1 - ssyt2, where ssyt1 and ssyt2 are tuples of column indexes.
def minus_ssyt(ssyt1, ssyt2):
    ssyt1 = [columns[i] for i in ssyt1]
    ssyt2 = [columns[i] for i in ssyt2]

    sub_ssyt = [[0 for _ in range(n)] for _ in range(len(ssyt1) - len(ssyt2))]
    for row in range(n):
        idx1 = 0
        idx2 = 0
        for i in range(len(ssyt1)-len(ssyt2)):
            while idx2 < len(ssyt2) and ssyt1[idx1][row] == ssyt2[idx2][row]:
                idx1 += 1
                idx2 += 1
            sub_ssyt[i][row] = ssyt1[idx1][row]
            idx1 += 1
    return tuple(column_idx[tuple(col)] for col in sub_ssyt)

# A cluster seed is a tuple (B, labels) where B is the extended adjacency matrix and labels
# are the ssyt for the non-frozen variables, represented as ssyt indexes.
l = m-n-1

frozen_labels = []
for i in range(m):
    column = column_idx[tuple(sorted([(i+j)%m for j in range(n)]))]
    ssyt = tuple([column])
    frozen_labels.append(add_ssyt(ssyt))
frozen_labels = tuple(frozen_labels)

cluster_labels = []
for r in range(l):
    for c in range(n-1, 0, -1):
        column = column_idx[tuple(list(range(c)) + list(range(c+1+r, n+1+r)))]
        ssyt = tuple([column])
        cluster_labels.append(add_ssyt(ssyt))
cluster_labels = tuple(cluster_labels)
num_cluster = len(cluster_labels)
num_frozen = len(frozen_labels)


# Define the extended adjacency matrix B.
B = [[0] * num_cluster for _ in range(num_cluster + num_frozen)]
# cluster to cluster edges
for r in range(l):
    # (r,c) <- (r,c+1)
    for c in range(n-2):
        i = r*(n-1) + c
        j = r*(n-1) + (c+1)
        B[j][i] = 1
        B[i][j] = -1
    # (r,c)
    #   ^
    #   |
    # (r+1, c)
    for c in range(n-1):
        if r < l-1:
            i = r*(n-1)+c
            j = (r+1)*(n-1)+c
            B[j][i] = 1
            B[i][j] = -1
    #(r,c)
    #    \
    #     > (r+1,c+1)
    for c in range(n-2):
        if r < l-1:
            i = (r+1)*(n-1)+(c+1)
            j = r*(n-1)+c
            B[j][i] = 1
            B[i][j] = -1
# frozen to cluster edges
B[num_cluster][0] = -1
for i in range(0, l+1):
    if i < l:
        B[num_cluster+1+i][i*(n-1) + (n-2)] = 1
    if i > 0:
        B[num_cluster+1+i][(i-1)*(n-1) + (n-2)] = -1
for i in range(l+1, m-1):
    B[num_cluster+1+i][(l-1)*(n-1)+(m-2-i)] = 1
    if i < m-2:
        B[num_cluster+1+i][(l-1)*(n-1)+(m-2-i)-1] = -1
B = tuple([tuple(row) for row in B])

# Store just the cluster labels for determining if we've reached a given seed
# before or not.
seen_seeds = set()
seen_seeds.add(cluster_labels)

# Each seed also stores the index of the most recently mutated variable.
# Due to commutativity of some mutations, we always try to mutate the lowest indexed
# variable first.
initial_seed = (cluster_labels, frozen_labels, B, -1)

# BFS Mutation.
todo = [initial_seed]
while len(todo) > 0:
    curr = todo.pop()
    cluster_labels, _, B, previous_mutation_index = curr
    for C in range(num_cluster):
        # By commutativity, we could have mutated C first and then previous_mutation_index
        if C <= previous_mutation_index and B[previous_mutation_index][C] == 0:
            continue

        in_variables = []
        out_variables = []

        for i in range(num_cluster + num_frozen):
            if B[i][C] > 0:
                in_variables.append(i)
            if B[i][C] < 0:
                out_variables.append(i)

        # Check the resulting rank before mutating.
        rank = 0
        for i in in_variables:
            if i < num_cluster:
                rank += len(ssyts[cluster_labels[i]])
            else:
                rank += len(ssyts[frozen_labels[i - num_cluster]])
        rank -= len(ssyts[cluster_labels[C]])
        # Don't mutate if the resulting rank would exceed `max_rank`.
        if rank > max_rank:
            continue

        # Mutate seed.
        ssyt_in = []
        ssyt_out = []
        for i in in_variables:
            if i < num_cluster:
                # ssyt_in = union_ssyt(ssyt_in, ssyts[cluster_labels[i]])
                ssyt_in.append(repeat_ssyt(ssyts[cluster_labels[i]], abs(B[i][C])))
            else:
                # ssyt_in = union_ssyt(ssyt_in, ssyts[frozen_labels[i - num_cluster]])
                ssyt_in.append(repeat_ssyt(ssyts[frozen_labels[i - num_cluster]], abs(B[i][C])))
        for i in out_variables:
            if i < num_cluster:
                ssyt_out.append(repeat_ssyt(ssyts[cluster_labels[i]], abs(B[i][C])))
                # ssyt_out = union_ssyt(ssyt_out, ssyts[cluster_labels[i]])
            else:
                # ssyt_out = union_ssyt(ssyt_out, ssyts[frozen_labels[i - num_cluster]])
                ssyt_out.append(repeat_ssyt(ssyts[frozen_labels[i - num_cluster]], abs(B[i][C])))
        ssyt_in = union_ssyt(*ssyt_in)
        ssyt_out = union_ssyt(*ssyt_out)
        
        weight_in = [0] * (n-1)
        weight_out = [0] * (n-1)
        for i in range(n-1):
            for c in ssyt_in:
                weight_in[i] += column_weights[c][i]
            for c in ssyt_out:
                weight_out[i] += column_weights[c][i]
        if all([weight_in[i] >= weight_out[i] for i in range(n-1)]) and any([weight_in[i] > weight_out[i] for i in range(n-1)]):
            new_ssyt = ssyt_in
        elif all([weight_in[i] <= weight_out[i] for i in range(n-1)]) and any([weight_in[i] < weight_out[i] for i in range(n-1)]):
            new_ssyt = ssyt_out
        else:
            # Useful debug information
            print(f"C = {C}; ssyts[C] = {[columns[c] for c in ssyts[cluster_labels[C]]]}")
            print(f"In variables: {in_variables}")
            print(f"In ssyts:")
            for i in in_variables:
                if i < num_cluster:
                    print([columns[c] for c in ssyts[cluster_labels[i]]])
                else:
                    print([columns[c] for c in ssyts[frozen_labels[i-num_cluster]]])
            print(f"In ssyt = {[columns[c] for c in ssyt_in]}")
            print(f"Out variables: {out_variables}")
            print(f"Out ssyts:")
            for i in out_variables:
                if i < num_cluster:
                    print([columns[c] for c in ssyts[cluster_labels[i]]])
                else:
                    print([columns[c] for c in ssyts[frozen_labels[i-num_cluster]]])
            print(f"Out ssyt = {[columns[c] for c in ssyt_out]}")
            print(C, weight_in, weight_out)
            raise ValueError(f"max({ssyt_in}, {ssyt_out}) is not well-defined!")
        
        new_ssyt = minus_ssyt(new_ssyt, ssyts[cluster_labels[C]])
        
        # Did not obtain a new SSYT.
        # if new_ssyt in ssyts:
        #     continue
        
        new_ssyt = add_ssyt(new_ssyt)
        new_cluster_labels = tuple([cluster_labels[i] if i != C else new_ssyt for i in range(num_cluster)])
        
        # We mutated back to a previous seed.
        if new_cluster_labels in seen_seeds:
            continue
        seen_seeds.add(new_cluster_labels)

        new_B = [list(B[r]) for r in range(num_cluster+num_frozen)]
        for i in in_variables:
            for j in out_variables:
                amt = B[i][C] * -B[j][C]
                if j < num_cluster:
                    new_B[i][j] += amt
                if i < num_cluster:
                    new_B[j][i] -= amt
        for i in in_variables + out_variables:
            new_B[i][C] *= -1
            if i < num_cluster:
                new_B[C][i] *= -1
        new_B = tuple(tuple(row) for row in new_B)

        todo.append((new_cluster_labels, frozen_labels, new_B, C))

print(f"#seen seeds = {len(seen_seeds)}")
print(f"#ssyts = {len(ssyts)}")